### Ridge Regression

In [9]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import RidgeCV
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler

In [10]:
df = pd.read_csv("merged_df.csv")
df.describe()

,Store,DayOfWeek,Sales,Customers,Open,Promo,SchoolHoliday,StateHoliday_encoded,isHoliday,CompetitionDistance,CompetitionOpenSinceMonth,CompetitionOpenSinceYear,Promo2,Promo2SinceWeek,Promo2SinceYear,StoreType_encoded,Assortment_encoded,PromoInterval_encoded
count,1.017209e+06,1.017209e+06,1.017209e+06,1.017209e+06,1.017209e+06,1.017209e+06,1.017209e+06,1.017209e+06,1.017209e+06,1.017209e+06,1.017209e+06,1.017209e+06,1.017209e+06,1.017209e+06,1.017209e+06,1.017209e+06,1.017209e+06,1.017209e+06
mean,5.584297e+02,3.998341e+00,5.773819e+03,6.331459e+02,8.301067e-01,3.815145e-01,1.786467e-01,4.516279e-02,8.213533e-01,5.422021e+03,7.336981e+00,2.008925e+03,5.005638e-01,1.164767e+01,1.007011e+03,1.207467e+00,9.351412e-01,3.093396e-01
std,3.219087e+02,1.997391e+00,3.849926e+03,4.644117e+02,3.755392e-01,4.857586e-01,3.830564e-01,2.836559e-01,3.830564e-01,7.706918e+03,2.674456e+00,4.999370e+00,4.999999e-01,1.532393e+01,1.005877e+03,1.365376e+00,9.938011e-01,1.421467e+00
min,1.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,2.000000e+01,1.000000e+00,1.900000e+03,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,-1.000000e+00
25%,2.800000e+02,2.000000e+00,3.727000e+03,4.050000e+02,1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.000000e+00,7.100000e+02,6.000000e+00,2.008000e+03,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,-1.000000e+00
50%,5.580000e+02,4.000000e+00,5.744000e+03,6.090000e+02,1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.000000e+00,2.325000e+03,8.000000e+00,2.010000e+03,1.000000e+00,1.000000e+00,2.009000e+03,0.000000e+00,0.000000e+00,1.000000e+00
75%,8.380000e+02,6.000000e+00,7.856000e+03,8.370000e+02,1.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00,1.000000e+00,6.880000e+03,9.000000e+00,2.011000e+03,1.000000e+00,2.200000e+01,2.012000e+03,3.000000e+00,2.000000e+00,1.000000e+00
max,1.115000e+03,7.000000e+00,4.155100e+04,7.388000e+03,1.000000e+00,1.000000e+00,1.000000e+00,3.000000e+00,1.000000e+00,7.586000e+04,1.200000e+01,2.015000e+03,1.000000e+00,5.000000e+01,2.015000e+03,3.000000e+00,2.000000e+00,3.000000e+00


In [11]:
# Convert date fields to datetime objects
df['Date'] = pd.to_datetime(df['Date'])
df['CompetitionOpenSinceDate'] = pd.to_datetime(df['CompetitionOpenSinceDate'])

# Calculate the duration of competition being open in months
df['CompetitionDurationMonths'] = (
    (df['Date'].dt.year - df['CompetitionOpenSinceDate'].dt.year) * 12 +
    (df['Date'].dt.month - df['CompetitionOpenSinceDate'].dt.month)
)

# Replace NaNs in 'CompetitionDurationMonths' with the median duration
df['CompetitionDurationMonths'].fillna(df['CompetitionDurationMonths'].median(), inplace=True)

# Drop the original date fields and other non-numeric or unnecessary columns
df = df.drop(['Date', 'CompetitionOpenSinceDate', 'Promo2Date'], axis=1)

# Convert categorical variables to dummy variables
df = pd.get_dummies(df, columns=['DayOfWeek', 'StateHoliday', 'StoreType', 'Assortment', 'PromoInterval'])

In [12]:
# drop sales and customers as they should not be part of the model
X = df.drop(['Sales', 'Customers'], axis=1)

# Define a range of alpha values to search
alphas = [0.1, 1, 10, 100]  # Add more values based on your requirements

In [13]:
# Define target for Sales
y_sales = df['Sales']

# Split the data for Sales
X_train_sales, X_test_sales, y_train_sales, y_test_sales = train_test_split(X, y_sales, test_size=0.2, random_state=42)

# Standardize the features for Sales
scaler_sales = StandardScaler()
X_train_sales_scaled = scaler_sales.fit_transform(X_train_sales)
X_test_sales_scaled = scaler_sales.transform(X_test_sales)

# Create and fit RidgeCV model for Sales using standardized features
ridge_cv_model_sales = RidgeCV(alphas=alphas, store_cv_values=True)
ridge_cv_model_sales.fit(X_train_sales_scaled, y_train_sales)

# Access the optimal alpha for Sales
optimal_alpha_sales = ridge_cv_model_sales.alpha_
print(f'Optimal Alpha for Sales: {optimal_alpha_sales}')

# Predict on the test set for Sales
y_pred_sales = ridge_cv_model_sales.predict(X_test_sales_scaled)

# Evaluate performance for Sales
mse_sales = mean_squared_error(y_test_sales, y_pred_sales)
rmse_sales = np.sqrt(mse_sales)
rmspe_sales = rmse_sales / np.mean(y_test_sales) * 100

print(f'Mean Squared Error for Sales: {mse_sales}')
print(f'Root Mean Squared Error for Sales: {rmse_sales}')
print(f'Root Mean Squared Percentage Error for Sales: {rmspe_sales}')

Optimal Alpha for Sales: 0.1
Mean Squared Error for Sales: 6184401.960422605
Root Mean Squared Error for Sales: 2486.845785412237
Root Mean Squared Percentage Error for Sales: 43.16206982958382


In [14]:
# Define features and target for Customers
y_customers = df['Customers']

# Split the data for Customers
X_train_customers, X_test_customers, y_train_customers, y_test_customers = train_test_split(X, y_customers, test_size=0.2, random_state=42)

# Standardize the features for Customers
scaler_customers = StandardScaler()
X_train_customers_scaled = scaler_customers.fit_transform(X_train_customers)
X_test_customers_scaled = scaler_customers.transform(X_test_customers)

# Create and fit RidgeCV model for Customers using standardized features
ridge_cv_model_customers = RidgeCV(alphas=alphas, store_cv_values=True)
ridge_cv_model_customers.fit(X_train_customers_scaled, y_train_customers)

# Access the optimal alpha for Customers
optimal_alpha_customers = ridge_cv_model_customers.alpha_
print(f'Optimal Alpha for Customers: {optimal_alpha_customers}')

# Predict on the test set for Customers
y_pred_customers = ridge_cv_model_customers.predict(X_test_customers_scaled)

# Evaluate performance for Customers
mse_customers = mean_squared_error(y_test_customers, y_pred_customers)
rmse_customers = np.sqrt(mse_customers)
rmspe_customers = rmse_customers / np.mean(y_test_customers) * 100

print(f'Mean Squared Error for Customers: {mse_customers}')
print(f'Root Mean Squared Error for Customers: {rmse_customers}')
print(f'Root Mean Squared Percentage Error for Customers: {rmspe_customers}')

Optimal Alpha for Customers: 0.1
Mean Squared Error for Customers: 91593.92623335756
Root Mean Squared Error for Customers: 302.6448846971605
Root Mean Squared Percentage Error for Customers: 47.88234197649414
